# 1. Clean file raw_belsimpel.xlsx

## a. import the file

In [11]:
import pandas as pd

bel_dataframe = pd.read_excel('/content/raw_belsimpel.xlsx') # Read an excel file

print(bel_dataframe.head()) # Print the first five rows of the dataframe


  web-scraper-order                    web-scraper-start-url       Rating  \
0      1674825213-2  https://www.belsimpel.nl/google-pixel-7    5 sterren   
1      1674825213-3  https://www.belsimpel.nl/google-pixel-7    5 sterren   
2      1674825213-4  https://www.belsimpel.nl/google-pixel-7  4.5 sterren   
3      1674825213-5  https://www.belsimpel.nl/google-pixel-7    5 sterren   
4      1674825213-6  https://www.belsimpel.nl/google-pixel-7  4.5 sterren   

                                         Review Date  \
0  *fd | 20-10-2022 over de Google Pixel 7 128GB ...   
1  *fd | 18-10-2022 over de Google Pixel 7 128GB ...   
2  *fd | 17-10-2022 over de Google Pixel 7 128GB ...   
3  *fd | 14-10-2022 over de Google Pixel 7 128GB Wit   
4  *fd | 27-10-2022 over de Google Pixel 7 128GB ...   

                                         Review Text  \
0  Vanaf een Huawei P30 pro die kapot ging de ove...   
1  Voor de pure Android liefhebber is het een uit...   
2  Na jaren geleden 2x een Nexus

## b. checking NULL values and replace with empty string

In [12]:
bel_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   web-scraper-order      486 non-null    object 
 1   web-scraper-start-url  486 non-null    object 
 2   Rating                 486 non-null    object 
 3   Review Date            486 non-null    object 
 4   Review Text            253 non-null    object 
 5   Pro_Con Review         372 non-null    object 
 6   Belsimpel              0 non-null      float64
dtypes: float64(1), object(6)
memory usage: 26.7+ KB


In [13]:
bel_dataframe['Pro_Con Review'].fillna("", inplace=True)
bel_dataframe['Review Text'].fillna("", inplace=True)

In [14]:
bel_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   web-scraper-order      486 non-null    object 
 1   web-scraper-start-url  486 non-null    object 
 2   Rating                 486 non-null    object 
 3   Review Date            486 non-null    object 
 4   Review Text            486 non-null    object 
 5   Pro_Con Review         486 non-null    object 
 6   Belsimpel              0 non-null      float64
dtypes: float64(1), object(6)
memory usage: 26.7+ KB


## c. Extract Rating from the raw rating data

In [15]:
import math
def extract_rating(st):
  return math.ceil(float(st.split()[0]))

bel_dataframe['Rating'] = bel_dataframe['Rating'].apply(extract_rating)

## d. Extract Date from raw Date Data

In [16]:
import datetime

def extract_date(text):
  date_str = text.split('over de')[0].split()[-1]
  date_obj = datetime.datetime.strptime(date_str, '%d-%m-%Y')
  return date_obj.strftime('%Y-%m-%d 00:00:00')

bel_dataframe['Review Date'] = bel_dataframe['Review Date'].apply(extract_date)

## e. Extract Pro Cons Review from raw data

In [17]:
def clean_list(lst):
  return [x.strip() for x in lst if x]

def extract_pro_con(text):
  pro_text = ", ".join(clean_list(str(text).split('Pluspunt')[:-1] + [str(text).split('Pluspunt')[-1].split('Minpunt')[0]]))
  con_text = ", ".join(clean_list(str(text).split('Pluspunt')[-1].split('Minpunt')[1:]))
  if pro_text != "":
    pro_text = "Pros: " + pro_text
  if con_text != "":
    con_text = "'.'Cons: " + con_text
  return pro_text +  con_text


bel_dataframe['Pro_Con Review'] = bel_dataframe['Pro_Con Review'].apply(extract_pro_con)


## f. Adding the pro_con reviews to Review Text

In [18]:
bel_dataframe['Review Text'] =  bel_dataframe['Review Text'] +"'.'"+ bel_dataframe['Pro_Con Review']

## g. if both the review text and pro_con review are empty then we remove those row from the dataframe

In [19]:
bel_dataframe = bel_dataframe[bel_dataframe['Review Text'] != "'.'"]

## h. Adding the country and source column to the dataframe

In [20]:
bel_dataframe['Country'] = 'Netherlands'
bel_dataframe['Source'] = 'Belsimpel'

print(bel_dataframe.head()) # Print the dataframe to verify the new columns


  web-scraper-order                    web-scraper-start-url  Rating  \
0      1674825213-2  https://www.belsimpel.nl/google-pixel-7       5   
1      1674825213-3  https://www.belsimpel.nl/google-pixel-7       5   
2      1674825213-4  https://www.belsimpel.nl/google-pixel-7       5   
3      1674825213-5  https://www.belsimpel.nl/google-pixel-7       5   
4      1674825213-6  https://www.belsimpel.nl/google-pixel-7       5   

           Review Date                                        Review Text  \
0  2022-10-20 00:00:00  Vanaf een Huawei P30 pro die kapot ging de ove...   
1  2022-10-18 00:00:00  Voor de pure Android liefhebber is het een uit...   
2  2022-10-17 00:00:00  Na jaren geleden 2x een Nexus gehad te hebben ...   
3  2022-10-14 00:00:00  Ik had hiervoor al de Pixel 6, daar was niks m...   
4  2022-10-27 00:00:00  Pixel 7 nu een ruime week in bezit, en beval m...   

                                      Pro_Con Review  Belsimpel      Country  \
0  Pros: Scherm, Snelhei

## i. Manage the dataframe into desired format

In [21]:
belsimpel_dataframe = pd.concat([bel_dataframe['Country'],bel_dataframe['Source'], bel_dataframe['Rating'],bel_dataframe['Review Date'],bel_dataframe['Review Text']],axis=1)

In [22]:
belsimpel_dataframe.head()

,Country,Source,Rating,Review Date,Review Text
0,Netherlands,Belsimpel,5,2022-10-20 00:00:00,Vanaf een Huawei P30 pro die kapot ging de ove...
1,Netherlands,Belsimpel,5,2022-10-18 00:00:00,Voor de pure Android liefhebber is het een uit...
2,Netherlands,Belsimpel,5,2022-10-17 00:00:00,Na jaren geleden 2x een Nexus gehad te hebben ...
3,Netherlands,Belsimpel,5,2022-10-14 00:00:00,"Ik had hiervoor al de Pixel 6, daar was niks m..."
4,Netherlands,Belsimpel,5,2022-10-27 00:00:00,"Pixel 7 nu een ruime week in bezit, en beval m..."


# 2. Clean file raw_shopee_Sg.xlsx

## a. import file

In [25]:
import pandas as pd

dataframe = pd.read_excel('/content/raw_shopee_Sg.xlsx') # Read an excel file

print(dataframe.head()) # Print the first five rows of the dataframe

  web-scraper-order                              web-scraper-start-url  \
0    1677488306-223  https://shopee.sg/Apple-iPhone-14-i.442800909....   
1    1677488306-224  https://shopee.sg/Apple-iPhone-14-i.442800909....   
2    1677488306-225  https://shopee.sg/Apple-iPhone-14-i.442800909....   
3    1677488306-226  https://shopee.sg/Apple-iPhone-14-i.442800909....   
4    1677488306-227  https://shopee.sg/Apple-iPhone-14-i.442800909....   

                                              Rating  \
0  <svg enable-background="new 0 0 15 15" viewBox...   
1  <svg enable-background="new 0 0 15 15" viewBox...   
2  <svg enable-background="new 0 0 15 15" viewBox...   
3  <svg enable-background="new 0 0 15 15" viewBox...   
4  <svg enable-background="new 0 0 15 15" viewBox...   

                                     Review Date  \
0  2022-12-28 15:41 | Variation: Starlight,512GB   
1     2022-10-11 05:50 | Variation: Purple,128GB   
2       2023-01-07 13:49 | Variation: Blue,128GB   
3       20

In [26]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   web-scraper-order      30 non-null     object 
 1   web-scraper-start-url  30 non-null     object 
 2   Rating                 30 non-null     object 
 3   Review Date            30 non-null     object 
 4   Review Text            30 non-null     object 
 5   Shopee                 0 non-null      float64
dtypes: float64(1), object(5)
memory usage: 1.5+ KB


## b. Extract rating from raw data

In [27]:
from bs4 import BeautifulSoup

def extract_rating(html_text):
  html_text = str(html_text)
  soup = BeautifulSoup(html_text, 'html.parser')  # Parse the HTML text
  rating_icons = soup.find_all('svg', class_='icon-rating-solid--active')   # Find all SVG elements with the specified class
  rating_value = len(rating_icons)   # Get the length of the list to determine the rating value
  return rating_value

dataframe['Rating'] = dataframe['Rating'].apply(extract_rating)

## c. Extract review Date from raw data

In [28]:
import datetime
def extract_review_date(text):
  date_str = text.split(' | ')[0]
  date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M')
  return date_obj.strftime('%Y-%m-%d 00:00:00')

dataframe['Review Date'] = dataframe['Review Date'].apply(extract_review_date)

## d. Extract review Text from raw data

In [29]:
from bs4 import BeautifulSoup

def extract_review_text(html_text):
  html_text = str(html_text)
  soup = BeautifulSoup(html_text, 'html.parser')  # Parse the HTML text
  div_tags = soup.find_all('div')    # Find all <div> tags
  all_text = []
  for div_tag in div_tags:       # Extract text from each <div> tag
      text = div_tag.get_text(strip=True)
      if text:
          all_text.append(text)

  other_text = [text for text in soup.find_all(string=True, recursive=False) if text.strip()]   # Find all text nodes directly under the root node
  all_text.extend(other_text)    # Add the text outside of <div> tags to the list
  text = "'.'".join(all_text)
  return text

dataframe['Review Text'] = dataframe['Review Text'].apply(extract_review_text)

<ipython-input-29-bcc125fc898a>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, 'html.parser')  # Parse the HTML text


## e. Adding country and source column to dataframe

In [30]:
dataframe['Country'] = 'Singapore'
dataframe['Source'] = 'Shopee-SG'

print(dataframe.head())

  web-scraper-order                              web-scraper-start-url  \
0    1677488306-223  https://shopee.sg/Apple-iPhone-14-i.442800909....   
1    1677488306-224  https://shopee.sg/Apple-iPhone-14-i.442800909....   
2    1677488306-225  https://shopee.sg/Apple-iPhone-14-i.442800909....   
3    1677488306-226  https://shopee.sg/Apple-iPhone-14-i.442800909....   
4    1677488306-227  https://shopee.sg/Apple-iPhone-14-i.442800909....   

   Rating          Review Date  \
0       5  2022-12-28 00:00:00   
1       5  2022-10-11 00:00:00   
2       5  2023-01-07 00:00:00   
3       5  2023-01-29 00:00:00   
4       5  2023-01-29 00:00:00   

                                         Review Text  Shopee    Country  \
0  Performance:yet to try'.'Best Feature(s):size ...     NaN  Singapore   
1  Performance:⭐️⭐️⭐️⭐️⭐️'.'Best Feature(s):⭐️⭐️⭐...     NaN  Singapore   
2  Performance:👍👍👍'.'Best Feature(s):👍👍👍'.'Value ...     NaN  Singapore   
3  Delivery within 3-4 working days. Haven’t trie.

## f. Manage dataframe into desired format

In [31]:
shopee_dataframe = pd.concat([dataframe['Country'],dataframe['Source'], dataframe['Rating'],dataframe['Review Date'],dataframe['Review Text']],axis=1)

In [33]:
shopee_dataframe.head()

,Country,Source,Rating,Review Date,Review Text
0,Singapore,Shopee-SG,5,2022-12-28 00:00:00,Performance:yet to try'.'Best Feature(s):size ...
1,Singapore,Shopee-SG,5,2022-10-11 00:00:00,Performance:⭐️⭐️⭐️⭐️⭐️'.'Best Feature(s):⭐️⭐️⭐...
2,Singapore,Shopee-SG,5,2023-01-07 00:00:00,Performance:👍👍👍'.'Best Feature(s):👍👍👍'.'Value ...
3,Singapore,Shopee-SG,5,2023-01-29 00:00:00,Delivery within 3-4 working days. Haven’t trie...
4,Singapore,Shopee-SG,5,2023-01-29 00:00:00,much cheaper than official site! delivery was ...


# 3. Concatinate both dataframe

In [34]:
output_belsimpel_shopee_dataframe = pd.concat([belsimpel_dataframe,shopee_dataframe],axis=0)

# 4. Save the dataframe into excel format

In [35]:
output_belsimpel_shopee_dataframe.to_excel('output_file.xlsx', index=False)